In [1]:
#from casadi import *
import numpy as np

import sympy
from sympy import sin, cos, simplify
from sympy import symbols as syms
from sympy.matrices import Matrix
from sympy.utilities.lambdify import lambdastr

import matplotlib.pyplot as plt
from draw_class import Visualizer

# System State derivation
\begin{align}
x_c &:= \mbox{cart position in x direction in m} \\
\phi_1 &:= \mbox{angle of first pole in deg} \\
\phi_2 &:= \mbox{angle of second pole in deg} \\
\dot{x_c} &:= \mbox{cart velocity in x direction in m/s} \\
\dot{\phi}_1 &:= \mbox{angular velocity of first pole in deg/s} \\
\dot{\phi}_2 &:= \mbox{angular of second pole in deg/s}
\end{align}

In [3]:
# init symbolic variables needed to describe the system
x_c, phi_1, phi_2, xdot_c, phidot_1, phidot_2, xddot_c, phiddot_1, phiddot_2, u = syms('x_c phi_1 phi_2 xdot_c phidot_1 phidot_2 xddot_c phiddot_1 phiddot_2 u')


# vector q holds system positions
q = Matrix([x_c, phi_1, phi_2])   
# first time derivative of q 
qdot = Matrix([xdot_c, phidot_1, phidot_2])
# seconds time derivative of q 
qddot = Matrix([xddot_c, phiddot_1, phiddot_2]) 
# system paramters
# m_c, m_1, m_2, l_1, l_2, g = syms('m_c, m_1, m_2, l_1, l_2, g')
m_c = 0.6
m_1 = 0.2 
m_2 = 0.2
l_1 = 0.5
l_2 = 0.5
g = 9.81



In [4]:
np.sin(0.25*np.pi)

0.7071067811865475

## Kinematics
The carts center of mass position is given by:
\begin{align}
p_c &= \begin{bmatrix}
           x_c \\
           0 
         \end{bmatrix}
\end{align}
and the first poles position can be calculated by (the center of mass is half-way along the pole):
\begin{align}
p_1 &= p_c + \frac{l_1}{2} \begin{bmatrix}
           sin(\phi_1) \\
           cos(\phi_1)
         \end{bmatrix}.
\end{align}
The center of mass position of the second pole can be derived by following the first pole and considering both joint angles to find the second poles direction:
\begin{align}
p_2 &= p_c + l_1 \begin{bmatrix}
           sin(\phi_1) \\
           cos(\phi_1)
         \end{bmatrix} + \frac{l_2}{2} \begin{bmatrix}
                                           sin(\phi_1 + \phi_2) \\
                                           cos(\phi_1 + \phi_2)
                                        \end{bmatrix}.
\end{align}


In [5]:
# cart position
p_c = Matrix([x_c, 0])
# position first pole
p_1 = p_c + l_1/2 * Matrix([-sin(phi_1), cos(phi_1)])
# position second pole 
p_2 = p_c + l_1 * Matrix([-sin(phi_1), cos(phi_1)]) + l_2/2 * Matrix([-sin(phi_1+phi_2), cos(phi_1+phi_2)])


To calculate the velocity, time derivatives of all position equations are nescarry. Here we use: 
\begin{align}
\frac{\partial f(x)}{\partial t} = \frac{\partial f(x)}{\partial x} \frac{\partial x}{\partial t} = \frac{\partial f(x)}{\partial x} \dot{x}.
\end{align}



In [6]:
# cart velocity
v_c = p_c.jacobian(Matrix([x_c])) * Matrix([xdot_c])
# velocity first pole
v_1 = p_1.jacobian(Matrix([x_c, phi_1])) * Matrix([xdot_c, phidot_1])
# velocity second pole
v_2 = p_2.jacobian(Matrix([x_c, phi_1, phi_2])) * Matrix([xdot_c, phidot_1, phidot_2])


The kinetic energy is given by:
\begin{align}
K = \frac{1}{2} m v²
\end{align}

In [7]:
K_c = m_c * v_c.T*v_c / 2
K_1 = m_1 * v_1.T*v_1 / 2
K_2 = m_2 * v_2.T*v_2 / 2

The potention energy is given by the y coordinate of each system part ($y=0$ is our baseline):
\begin{align}
P = mgp_y
\end{align}

In [8]:
P_1 = Matrix([m_1 * g * p_1[1]])
P_2 = Matrix([m_2 * g * p_2[1]])

The kinetic and potential energy of the system is plugged into the Euler-Lagrange equation. 

*TODO: Rewrite var names and structure*

In [9]:
L = K_c + K_1 + K_2 - P_1 - P_2

# uirst term in the Euler-Lagrange equation
partial_L_by_partial_q = L.jacobian(Matrix([q])).T

# inner term ou the second part ou the Euler-Lagrange equation
partial_L_by_partial_qdot = L.jacobian(Matrix([qdot]))

# second term (overall, time derivative) in the Euler-Lagrange equation
# applies the chain rule
d_inner_by_dt = partial_L_by_partial_qdot.jacobian(Matrix([q])) * qdot + partial_L_by_partial_qdot.jacobian(Matrix([qdot])) * qddot

# Euler-Lagrange equation
lagrange_eq = partial_L_by_partial_q - d_inner_by_dt + Matrix([u,0,0])


Solve Euler Lagrange for $\ddot{x}_c$, $\ddot{\phi}_1$ and $\ddot{\phi}_2$

In [10]:
print("This will take some time...")
r = sympy.solvers.solve(simplify(lagrange_eq), Matrix([qddot]))


This will take some time...


In [11]:
print("This will also take some time...")
xddot_c_sol = simplify(r[xddot_c])
phiddot_1_sol = simplify(r[phiddot_1])
phiddot_2_sol = simplify(r[phiddot_2])

This will also take some time...


## Casadi


In [12]:
# casadi should be imported here to prevent overloading of sympy functions
from casadi import *

# redefine all sympy symbols as casadi symbols
x_c = SX.sym('x_c')
phi_1 = SX.sym('phi_1')
phi_2 = SX.sym('phi_2')
xdot_c = SX.sym('xdot_c')
phidot_1 = SX.sym('phidot_1')
phidot_2 = SX.sym('phidot_2')
xddot_c =SX.sym('xddot_c')
phiddot_1 = SX.sym('phiddot_1')
phiddot_2 = SX.sym('phiddot_2')
nu = 1
u = SX.sym('u', nu)

# create casadi function for each ODE this will make the reduction of order easier
xddot_c_cas_symb = eval(str(xddot_c_sol))
xddot_c_fun  = Function("xddot_c", [phi_1, phi_2, xdot_c, phidot_1, phidot_2, u], [xddot_c_cas_symb])
                   
phiddot_1_cas_symb = eval(str(phiddot_1_sol))
phiddot_1_fun  = Function("phiddot_1", [phi_1, phi_2, xdot_c, phidot_1, phidot_2, u], [phiddot_1_cas_symb])

phiddot_2_cas_symb = eval(str(phiddot_2_sol))
phiddot_2_fun  = Function("phiddot_2", [phi_1, phi_2, xdot_c, phidot_1, phidot_2, u], [phiddot_2_cas_symb])
    

Convert 2nd order ODEs to system of first order ODEs where:
\begin{align}
    x_1 &= x_c, \\
    x_2 &= \dot{x_c}, \\
    x_3 &= \phi_1, \\
    x_4 &= \dot{\phi_1}, \\
    x_5 &= \phi_2 \\
    x_6 &= \dot{\phi_2}
\end{align}

In [13]:
nx = 6

x = SX.sym("x", nx)

# create system of first order ODEs
xdot = np.array([[x[1]], # xdot_1
                 [xddot_c_fun(x[2], x[4], x[1], x[3], x[5], u)], # xdot_2
                 [x[3]], # xdot_3
                 [phiddot_1_fun(x[2], x[4], x[1], x[3], x[5], u)], # xdot_3
                 [x[5]], # xdot_4
                 [phiddot_2_fun(x[2], x[4], x[1], x[3], x[5], u)]]) # xdot_4

system = Function("system", [x, u], [xdot])